In [1]:

# So first take a look at http://glaciertools.net.s3.amazonaws.com/its-live-tool-maps/velOnlyItslive_3031_raster/openlayers.html
# Now if we want to use it in ipyleaflet ->

from ipyleaflet import (
    Map,
    basemaps,
    basemap_to_tiles,
    TileLayer,
    GeoJSON,
    DrawControl,
    LayersControl,
    projections
)

lc = LayersControl(position='topright')

# So we get what the map coords when we draw a polygon etc.
def handle_draw(target, action, geo_json):
    print(action, geo_json)

dc = DrawControl(marker={'shapeOptions': {'color': '#0000FF'}})
dc.on_draw(handle_draw)


# This is the definition that works with GIBS but it's applied to the entire map, in OpenLayers
# we can define them individually for each layer. I played with this to match what GDal2Tiles generated and still didn't get
# the right tile requests.
my_projection = {
    'name': 'EPSG:3031',
    'custom': True, #This is important, it tells ipyleaflet that this projection is not on the predefined ones.
    'proj4def': '+proj=stere +lat_0=-90 +lat_ts=-71 +lon_0=0 +k=1 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs',
    'origin': [-4194304, 4194304],
    'bounds': [
        [-4194304, -4194304],
        [4194304, 4194304]
    ],
    'resolutions': [
        16384.0,
        8192.0,
        4096.0,
        2048.0,
        1024.0,
        512.0,
        256.0
    ]
}

velocities_url = (
    'http://glaciertools.net.s3.amazonaws.com/its-live-tool-maps/velOnlyItslive_3031_raster/{z}/{x}/{y}.png'
)

# https://github.com/jupyter-widgets/ipyleaflet/blob/d9d09a74915531fd1845fa95d89a9419c2febfd2/ipyleaflet/leaflet.py#L521
tms = TileLayer(
    url=velocities_url,
    opacity=0.8,
    tile_size=512,
    tms=True,
    min_zoom=1,
    max_zoom=9,
    attribution=' geospatial'
)

antarctic_map = Map(center=(-90, 0),
                    zoom=1,
                    layers=(basemap_to_tiles(basemaps.NASAGIBS.BlueMarble3031),tms),
                    crs=my_projection)


antarctic_map.add_control(dc)
antarctic_map.add_control(lc)

# sometimes Firefox fails to handle the mouse in the map so you have to use the arrow keys to move around.

antarctic_map

Map(center=[-90, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…


At some point I also considered https://github.com/jupyter-widgets/ipyleaflet/blob/f47ace2a2a3c5b27ca1c0079d8e07bc9ab22e601/examples/CustomTileServer.ipynb

To correct the tile request on the fly. But didn't get too far either.
